# Author: John Njeri

## This is a webscrapping and mapping for Toronto, CA, with the aim of identifying clusters of neighborhoods and they most common venues

### The purpose is to enable individuals relocating to Toronto make a decision based on the amenities of different neughborhood clusters based on their preferences

### Importing the relevant libraries

In [2]:
#importing the relevant libraries
import requests
from bs4 import BeautifulSoup

### Getting the website using requests library and scrapping with BeautifuSoup

In [3]:
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web_page=requests.get(website_url)

In [3]:
print(web_page.status_code)

200


In [4]:
soup = BeautifulSoup(web_page.content, 'lxml')
#print(soup.prettify())

In [5]:
#Getting the HTML part that has the table
my_table=soup.find('table', {'class':'wikitable sortable'})


### Appending HTML variables to respective lists

In [6]:
#Appening the specific parts of the table(postcode, borough, and neighborhood) to respective lists
postcode=[]
borough=[]
neighborhood=[]
for row in my_table.find_all('tr')[1:]:
    cells = row.find_all('td')
    postcode.append(cells[0].text)
    borough.append(cells[1].text)
    neighborhood.append(cells[2].text)

In [67]:
#print(borough)
#print(len(postcode))
print(neighborhood)

Northwest


In [8]:
#Getting the clomun titles from the html page
for head in my_table.find_all('tr')[:1]:
    x=[head.text]
s_tr="".join(x)
headings=s_tr.strip('\n').split('\n')
headings

['Postcode', 'Borough', 'Neighbourhood']

In [9]:
#removing the \n section of the list and '' created from the empty list
neighborhood_edited="".join(neighborhood).split('\n')
neighborhood_edited.pop()

''

In [10]:
print(len(neighborhood_edited))

288


#### Creating a Pandas DataFrame using the variables above

In [11]:
#Putting these parts into a dataframe
import pandas as pd
import numpy as np
df = pd.DataFrame(columns=headings)

In [12]:
df['Postcode']=postcode
df['Borough']=borough
df['Neighborhood']=neighborhood_edited

### Removing null variables and joining replicated rows

In [68]:
#using .join to remove duplicate variables
new_df=df[df.Borough != 'Not assigned']
neighborhood_df=new_df.groupby('Postcode').Neighborhood.agg([('Neighborhood', ', '.join)])
neighborhood_df.head(10)

,Neighborhood
Postcode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae
M1J,Scarborough Village
M1K,"East Birchmount Park, Ionview, Kennedy Park"
M1L,"Clairlea, Golden Mile, Oakridge"
M1M,"Cliffcrest, Cliffside, Scarborough Village West"


In [14]:
# joining adjacent columns
sample_df=new_df.groupby('Postcode').Borough.agg([('Borough', ', '.join)])
sample_df['borough']=[x.split(',')[0] for x in sample_df['Borough']]
sample_df.drop('Borough', axis=1, inplace=True)

In [15]:
#joining adjacent columns
final_df = pd.merge(sample_df, neighborhood_df, on='Postcode').reset_index()
final_df.head(10)

,Postcode,borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
### Replacing none assigned variables in Neighborhood columns with corresponding variables in the borough column

In [17]:
final_df[final_df['Neighborhood']=='Not assigned']

,Postcode,borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [18]:
final_df.loc[85,'Neighborhood']=final_df.loc[85,'borough']

In [19]:
final_df.shape

(103, 3)

## Geolocation part (Matching the coordinate to the relevant postocde)

In [20]:
#importing the relevant libraries
import folium

In [21]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
geo_data.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
geo_data.head(5)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
combined_df=pd.merge(final_df, geo_data, on='Postcode', how='inner')
combined_df.head(10)

,Postcode,borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [24]:
len(combined_df)

103

## Data Exploration with maps

In [25]:
#Importing dependencies for exploring the neighborhoods in maps
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [26]:
#Getting the json file with respective latittude and longitude for Toronto, Ontario, Canada
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [27]:
#Creating the map of Toronto with the neighborhoods superimposed on top
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(combined_df['Latitude'], combined_df['Longitude'], combined_df['borough'], combined_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [69]:
#Getting a dataframe that has the word Toronto
Toronto_df = combined_df[combined_df['borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_df.head(10)

,Postcode,borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


### Getting the neighborhoods in Toronto for the new dataframe

In [29]:
# create map of Manhattan using latitude and longitude values
map_toronto_neighborhoods = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighborhoods)  
    
map_toronto_neighborhoods

In [30]:
# @hidden_cell
#Exploring the neighborhoods with Foursquare API
CLIENT_ID = 'NQPNDWETFTD24QGDXYVQ5TX0BMH5TWPJI2QSC2EEFKZEJ4HT' # your Foursquare ID
CLIENT_SECRET = 'RQTOOT15W0PVSDKQT3PWWXMPLDYQ0LFSRBAIKROEPR4XVGIB' # your Foursquare Secret
VERSION = '20190419' # Foursquare API version

#### Exploring University of Toronto which is at index 25(random)

In [31]:
#Exploring University of Toronto which is at index 25
Toronto_df.loc[25, 'Neighborhood']

'Harbord, University of Toronto'

In [32]:
#Getting the neighborhood's longitude and lattitude values and name
neighborhood_lat = Toronto_df.loc[25, 'Latitude'] # neighborhood latitude value
neighborhood_long = Toronto_df.loc[25, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_df.loc[25, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude values of Harbord, University of Toronto are 43.6626956, -79.4000493.


In [33]:
#Getting the top 100 venues within 500 meters of the neighborhood
limit = 100 #Number of venues returned by the foursquare API
radius_neigh = 500 #defining the radius

#Creating the url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_long, 
    radius_neigh, 
    limit)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=NQPNDWETFTD24QGDXYVQ5TX0BMH5TWPJI2QSC2EEFKZEJ4HT&client_secret=RQTOOT15W0PVSDKQT3PWWXMPLDYQ0LFSRBAIKROEPR4XVGIB&v=20190419&ll=43.6626956,-79.4000493&radius=500&limit=100'

In [34]:
#sending a get request and displaying the results
results_venues = requests.get(url).json()
results_venues

{'meta': {'code': 200, 'requestId': '5cba0c04351e3d3ef1339325'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 43.6671956045, 'lng': -79.39384042790832},
   'sw': {'lat': 43.6581955955, 'lng': -79.4062581720917}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5362c366498e602fbe1db395',
       'name': 'Yasu',
       'location': {'address': '81 Harbord St.',
        'lat': 43.66283719650635,
        'lng': -79.40321739973975,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.662

In [35]:
#All information from the Foursquare API is contained in the item key
#To extract the categories of the venues we get a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
#Creating the json file and flattening the json file
venues_loc = results_venues['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues_loc) # flatten JSON
#venues_loc
nearby_venues.head(3)

,reasons.count,reasons.items,referralId,venue.categories,venue.events.count,venue.events.items,venue.events.summary,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5362c366498e602fbe1db395-0,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",NaN,NaN,NaN,5362c366498e602fbe1db395,81 Harbord St.,CA,Toronto,Canada,NaN,255,"[81 Harbord St., Toronto ON M5S 1G4, Canada]","[{'label': 'display', 'lat': 43.66283719650635...",43.662837,-79.403217,"Harbord Village, Toronto, ON",M5S 1G4,ON,Yasu,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-56eb72e3498eefb98d82274b-1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,56eb72e3498eefb98d82274b,88 Harbord St.,CA,Toronto,Canada,Harbord & Spadina,231,"[88 Harbord St. (Harbord & Spadina), Toronto O...","[{'label': 'display', 'lat': 43.66294872979362...",43.662949,-79.402898,NaN,M5S 1G5,ON,Piano Piano,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-527d450111d25050de4ea0d8-2,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",NaN,NaN,NaN,527d450111d25050de4ea0d8,196 Robert Street,CA,NaN,Canada,NaN,317,"[196 Robert Street, Canada]","[{'label': 'display', 'lat': 43.66275675127544...",43.662757,-79.403988,NaN,NaN,NaN,Rasa,0,[],NaN


In [37]:
#Filtering the columns
filtered_cols = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_cols]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Yasu,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",43.662837,-79.403217
1,Piano Piano,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",43.662949,-79.402898
2,Rasa,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",43.662757,-79.403988
3,The Dessert Kitchen,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",43.662823,-79.402746
4,Almond Butterfly,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.662836,-79.403365


In [38]:
#Adding the venues categories in nearby_venues
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Piano Piano,Italian Restaurant,43.662949,-79.402898
2,Rasa,Restaurant,43.662757,-79.403988
3,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
4,Almond Butterfly,Bakery,43.662836,-79.403365


In [39]:
#Cleaning the columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Piano Piano,Italian Restaurant,43.662949,-79.402898
2,Rasa,Restaurant,43.662757,-79.403988
3,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
4,Almond Butterfly,Bakery,43.662836,-79.403365


In [40]:
#the number of venues returned by Foursquare is as follows
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

35 venues were returned by Foursquare.


### After exploring one neighborhood we can try exploring all the 33 venues retuened by Foursquare

In [42]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    list_of_venues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url_1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results_1 = requests.get(url_1).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        list_of_venues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_1])

    nearby_venues_1 = pd.DataFrame([item for venue_list in list_of_venues for item in venue_list])
    nearby_venues_1.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_1)

In [43]:
#code that runs the above function on each neighborhood and creates a new dataframe by the name Toronto_venues_1
Toronto_venues_1 = getVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [44]:
print(Toronto_venues_1.shape)
Toronto_venues_1.head()

(1721, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [45]:
#Number of venues returned for each neighborhood
Toronto_venues_1.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [46]:
#Getting the unique categories from the above venues
print('There are {} uniques categories.'.format(len(Toronto_venues_1['Venue Category'].unique())))

There are 236 uniques categories.


#### Analyzing each neighborhood

In [47]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues_1[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues_1['Neighborhood'] 

# move neighborhood column to the first column
new_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[new_columns]

Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [48]:
Toronto_onehot.shape

(1721, 236)

In [49]:
#grouping the rows by neighborhoods and the mean of the frequency of the occurence of each category
Toronto_df_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_df_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.04,0.0,0.0,0.0,0.010000,0.01,0.0,0.03,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.03,0.010000,0.0,0.0,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.010000,0.000000,0.060000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.0,0.000000,0.0,0.0,0.01,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000

In [50]:
#size of the new dataframe
Toronto_df_grouped.shape

(38, 236)

#### Getting the neighborhoods along with the top 5 most common venues

In [51]:
#Getting the neighborhoods along with the top 5 most common venues
num_of_venues = 5

for hood in Toronto_df_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp_venue = Toronto_df_grouped[Toronto_df_grouped['Neighborhood'] == hood].T.reset_index()
    temp_venue.columns = ['venue','freq']
    temp_venue = temp_venue.iloc[1:]
    temp_venue['freq'] = temp_venue['freq'].astype(float)
    temp_venue = temp_venue.round({'freq': 3})
    print(temp_venue.sort_values('freq', ascending=False).reset_index(drop=True).head(num_of_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
                venue   freq
0         Coffee Shop  0.088
1        Cocktail Bar  0.053
2      Farmers Market  0.035
3              Bakery  0.035
4  Seafood Restaurant  0.035


----Brockton, Exhibition Place, Parkdale Village----
                venue   freq
0      Breakfast Spot  0.091
1                Café  0.091
2         Coffee Shop  0.091
3  Italian Restaurant  0.045
4       Burrito Place  0.045


----Business Reply Mail Processing Centre 969 Eastern----
                venue   freq
0  Light Rail Station  0.105
1         Yoga Studio  0.053
2         Pizza Place  0.053
3             Brewery  0.053
4          Smoke Shop  0.053


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              ven

In [53]:
#A function that returns the venues in a descending order
def common_venues(row, num_common_venues):
    row_categories = row.iloc[1:]
    row_categories_desc = row_categories.sort_values(ascending=False)
    
    return row_categories_desc.index.values[0:num_common_venues]

#### dataframe that Gives the top 10 venues for each neighborhood

In [54]:
#A dataframe that Gives the top 10 venues for each neighborhood
num_common_venues = 10

premise_indicator = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for indicator in np.arange(num_common_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(indicator+1, premise_indicator[indicator]))
    except:
        columns.append('{}th Most Common Venue'.format(indicator+1))

# create a new dataframe
neighborhoods_venues_desc = pd.DataFrame(columns=columns)
neighborhoods_venues_desc['Neighborhood'] = Toronto_df_grouped['Neighborhood']

for indicator in np.arange(Toronto_df_grouped.shape[0]):
    neighborhoods_venues_desc.iloc[indicator, 1:] = common_venues(Toronto_df_grouped.iloc[indicator, :], num_common_venues)

neighborhoods_venues_desc.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Cheese Shop,Restaurant,Bakery,Farmers Market,Steakhouse,Café,Seafood Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Recording Studio,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport


### Clustering the neighborhoods with 5 clusters

In [55]:
#initiating 5 clusters
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [56]:
#A Dataframe that has the cluster and the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_desc.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_desc.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postcode,borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Health Food Store,Pub,Wings Joint,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Liquor Store,Bakery,Sports Bar,Spa
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Pizza Place,Pet Store,Food & Drink Shop,Liquor Store,Light Rail Station,Burger Joint,Burrito Place,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Sandwich Place,Stationery Store,Bank,Bar,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Swim School,Wings Joint,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Visualizing the clusters with a map

In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat_1, lon_1, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(poi + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat_1, lon_1],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

#### we can identifying the discriminating venue categories that distinguish each cluster

#### First Cluster

In [60]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Health Food Store,Pub,Wings Joint,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Liquor Store,Bakery,Sports Bar,Spa
2,East Toronto,0,Park,Sandwich Place,Pizza Place,Pet Store,Food & Drink Shop,Liquor Store,Light Rail Station,Burger Joint,Burrito Place,Fast Food Restaurant
3,East Toronto,0,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Sandwich Place,Stationery Store,Bank,Bar,Fish Market
5,Central Toronto,0,Park,Clothing Store,Sandwich Place,Food & Drink Shop,Hotel,Burger Joint,Asian Restaurant,Breakfast Spot,Gym,Donut Shop
6,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Gift Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Mexican Restaurant,Bagel Shop,Park
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Gym,Toy / Game Store,Gourmet Shop
9,Central Toronto,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station
11,Downtown Toronto,0,Coffee Shop,Restaurant,Pizza Place,Italian Restaurant,Café,Pub,Bakery,Park,Farmers Market,Japanese Restaurant
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Burger Joint,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant


#### Second Cluster

In [61]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Gym,Playground,Grocery Store,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Third Cluster

In [62]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Fourth Cluster

In [63]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Park,Playground,Trail,Wings Joint,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
23,Central Toronto,3,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Fifth Cluster

In [64]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Bus Line,Park,Swim School,Wings Joint,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Based on the It would be easy for a customer who is relocating to identify places that have similar amenities that were in the previous place or even amenities that they desire. Additionally, new individuals moving into Toronto can easily identify a place that suits their preferences.